# Imports

In [172]:
import pandas as pd
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
import json
import re
import csv

## Read pos and neg lines
- There are around ~30k tweets for positive and negative commentaries
- Twitter api only supports 100 tweets per request
- Here, we separe all tweets in groups of 100

In [99]:
pos_lines_without_n = []
with open('./positivos.txt') as f:
    lines = f.readlines()

for line in lines:
    pos_lines_without_n.append(line.strip('\n'))

counter = 0;
pos_lines = []
aux = []
for line in pos_lines_without_n:
    if(counter < 100):
        aux.append(line)
        counter += 1
    else:
        pos_lines.append(aux)
        aux = []
        counter = 0

In [101]:
neg_lines_without_n = []
with open('./negativos.txt') as f:
    lines = f.readlines()

for line in lines:
    neg_lines_without_n.append(line.strip('\n'))

counter = 0;
neg_lines = []
aux = []
for line in neg_lines_without_n:
    if(counter < 100):
        aux.append(line)
        counter += 1
    else:
        neg_lines.append(aux)
        aux = []
        counter = 0

## Load twitter credentials

In [103]:
with open('../twitter-api/twitter_credentials.json', 'r') as file:
    creds = json.load(file)

auth = OAuthHandler(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])
auth.set_access_token(creds['ACCESS_TOKEN'], creds['ACCESS_KEY'])
api = API(auth)

## Fetch positive and negative tweets

In [155]:
tweets = []
for tweet_list in pos_lines:
    aux = api.statuses_lookup(tweet_list)
    for single_tweet in aux:
        tweets.append(single_tweet)
pos_tweets = tweets

In [128]:
tweets = []
for tweet_list in neg_lines:
    aux = api.statuses_lookup(tweet_list)
    for single_tweet in aux:
        tweets.append(single_tweet)
#     tweets = (api.statuses_lookup(pos_lines[0]))
neg_tweets = tweets

In [158]:
print(len(neg_tweets))
print(len(pos_tweets))

27098
28929


## Filter tweets

In [191]:
filtred_pos_tweets = []
for idx, tweet in enumerate(pos_tweets):
        aux = re.sub(r'http\S+', '', tweet.text)
        aux = aux.replace(':)', '')
        aux = aux.replace(':-)', '')
        aux = re.sub(r'@\S+', '', aux)
        aux = re.sub(r'#\S+', '', aux)
        aux = ' '.join(aux.split())
        filtred_pos_tweets.append(aux)

In [192]:
filtred_neg_tweets = []
for idx, tweet in enumerate(neg_tweets):
        aux = re.sub(r'http\S+', '', tweet.text)
        aux = aux.replace(':(', '')
        aux = aux.replace(':-(', '')
        aux = re.sub(r'@\S+', '', aux)
        aux = re.sub(r'#\S+', '', aux)
        aux = ' '.join(aux.split())
        filtred_neg_tweets.append(aux)

## Opens csv files and load writes

In [188]:
csv_file = open('pos_tweets.csv', 'a')
csv_neg_file = open('neg_tweets.csv', 'a')
csv_writer = csv.writer(csv_file)
csv_neg_writer = csv.writer(csv_neg_file)

In [174]:
for tweet in filtred_pos_tweets:
    csv_writer.writerow([tweet])

In [189]:
for tweet in filtred_neg_tweets:
    csv_neg_writer.writerow([tweet])